<a href="https://colab.research.google.com/github/AlexMcLaughlin1/Shoe-Shop-Application-OOP-SQL-Database-/blob/main/shoe_shop_application.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sqlite3

connection = sqlite3.connect('shoe_shop.db')
cursor = connection.cursor()
cursor.execute('''CREATE TABLE IF NOT EXISTS Customers
               (
                 unique_id int,
                 username text(100),
                 password text(25),
                 delivery_address text(200),
                 contact_info int,
                 bank_detail int, 
                 max_returns_per_month int
                 )             
               '''
               )
connection.commit()
connection.close()

In [ ]:
connection = sqlite3.connect('shoe_shop.db')
cursor = connection.cursor()
cursor.execute('''CREATE TABLE IF NOT EXISTS Stock
               (
                 shoe text(100),
                 stock int
                 )             
               '''
               )
connection.commit()
connection.close()

In [ ]:
connection = sqlite3.connect('shoe_shop.db')
cursor = connection.cursor()
cursor.execute('''CREATE TABLE IF NOT EXISTS Orders
               (
                 Order_Number text(20),
                 Customer_ID int,
                 Order_Date DATETIME,
                 Delivery_Address text(200),
                 Latest_Delivery_Date DATETIME,
                 Product text(100),
                 Quantity int,
                 Free_Return_Date text(100),
                 Returned int ,
                 Returned_Date DATETIME
                 )             
                '''
               )
connection.commit()
connection.close()

In [ ]:
class Shoes:

  def __init__(self, name, price, stock=100):
    self.name = name
    self.stock = stock
    self.price = price
    self.add_to_db()

  def add_to_db(self):
    shoe_info = (self.name, self.stock)
    connection = sqlite3.connect('shoe_shop.db')
    cursor = connection.cursor()
    cursor.execute('''
      INSERT INTO Stock
      VALUES (?, ?)
    ''',
    shoe_info
    )
    connection.commit()
    connection.close()

  def check_stock(self):
    connection = sqlite3.connect('shoe_shop.db')
    cursor = connection.cursor()
    cursor.execute('''
      SELECT stock
      FROM Stock
      WHERE shoe = ?
    ''',
    (self.name, )
    )
    return cursor.fetchone()[0]
  
  def update_stock_from_order(self, quantity):
    connection = sqlite3.connect('shoe_shop.db')
    cursor = connection.cursor()
    cursor.execute('''
      SELECT stock
      FROM Stock
      WHERE shoe = ?
    ''',
    (self.name, )
    )
    current_stock = cursor.fetchone()[0]


    cursor.execute('''
      UPDATE Stock
      SET stock = ?
      WHERE shoe = ?
    ''',
    (current_stock - quantity, self.name)
    )
    self.stock = current_stock - quantity ### 
    connection.commit()
    connection.close()

  def new_stock(self, quantity):
    connection = sqlite3.connect('shoe_shop.db')
    cursor = connection.cursor()
    cursor.execute('''
      SELECT stock
      FROM Stock
      WHERE shoe = ?
    ''',
    (self.name, )
    )
    current_stock = cursor.fetchone()[0]


    cursor.execute('''
      UPDATE Stock
      SET stock = ?
      WHERE shoe = ?
    ''',
    (current_stock + quantity, self.name)
    )
    self.stock = current_stock + quantity
    connection.commit()
    connection.close()

In [ ]:
sandles = Shoes('sandles', 20)
boaters = Shoes('boaters', 25)
boots = Shoes('boots', 40)
sliders = Shoes('sliders', 15, stock=50)
stilettos = Shoes('stilettos', 36)
clogs = Shoes('clogs', 23)
slippers = Shoes('slippers', 12, stock=200)
trainers = Shoes('trainers', 35)
moccasins = Shoes('moccasins', 10)
brogues = Shoes('brogues', 20)

In [ ]:
import itertools
ordernumiter = itertools.count()

In [ ]:
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import pandas as pd

class Customer(object):
  id_iter = itertools.count()  # creates unique id by counting numerically from 0 each time a new customer registers
  def __init__(self, username, password, delivery_address, contact_info, bank_detail):
    self.username = username
    self.password = password
    self.delivery_address = delivery_address
    self.contact_info = contact_info
    self.bank_detail = bank_detail
    self.unique_id = next(Customer.id_iter)
    if self.check_already_registered():
      self.add_to_db()
    self.loggedin = False
    


  def add_to_db(self):
    cust_info = (self.unique_id, self.username, self.password, self.delivery_address, self.contact_info, self.bank_detail, 2)
    connection = sqlite3.connect('shoe_shop.db')
    cursor = connection.cursor()
    cursor.execute('''
      INSERT INTO Customers
      VALUES (?, ?, ?, ?, ?, ?, ?)
    ''',
    cust_info
    )
    connection.commit()
    connection.close()

  def check_already_registered(self):
    connection = sqlite3.connect('shoe_shop.db')
    cursor = connection.cursor()
    cursor.execute('''
      SELECT username
      FROM Customers
      WHERE username = ?
    ''',
    (self.username, )
    )

    username_created = cursor.fetchall()

    if len(username_created) >= 1:
      print('Username already registered please login.')
    else:
      return True



  def login(self, username, password):
    if username != self.username:
      print('Username not recognised.')
      return
    else:
      if password != self.password:
        print('Incorrect Password Entered.')
        return
      else:
        print('Login successful')
        self.loggedin = True

  def logout(self):
    if self.loggedin:
      print('Logout Successful')
      self.loggedin = False
    else:
      print('Not logged in')


  def search(self, shoe_type, buy=False, quantity=0):
    if self.loggedin == True: # only logged in customers can search
      input = shoe_type.lower()
      regex = ['%', input, '%']
      regex = ''.join(regex) 
      # creates regualr expression in SQL form so any shoe which containes input shoe_type will be returned
      connection = sqlite3.connect('shoe_shop.db')
      cursor = connection.cursor()

      cursor.execute('''SELECT shoe, stock 
                        FROM stock
                        WHERE shoe LIKE ?
                        ''',
                    (regex,)
                    )
      # Above selects all shows from stock database which contain input

      shoes = cursor.fetchall()
      for row in shoes:
        print(row[0], '.'*(25-len(row[0])), 'stock:', row[1]) #prints all returned shoes and stock level

      if buy: # shoes can also be bought from search method
        connection = sqlite3.connect('shoe_shop.db')
        cursor = connection.cursor()
        cursor.execute('''
          SELECT *
          FROM stock
          '''
          ) # checks shoe_type is input as expected
        shoes = []
        for row in cursor.fetchall():
          shoes.append(row[0])
        if input in shoes:
          if quantity == 0:
            print('please select quantity')
          else:
            self.buy_shoes(shoe_type, quantity) # as long as quantity and shoe type entered correctly then calls buy_shoes method
        else: 
          print('Please type shoe type as listed to purchase.')

      connection.commit()
      connection.close()
    else:
      print('Please login to account.')




  def buy_shoes(self, shoe_type, quantity=1):
    
    if self.loggedin != True: #must be logged on to buy shoes
      print('Please login.')
      return
    
    else: # Next checks the shoe is entered correctly
      connection = sqlite3.connect('shoe_shop.db')
      cursor = connection.cursor()
      cursor.execute('''
        SELECT *
        FROM stock
        '''
        )
      shoes = []
      for row in cursor.fetchall():
        shoes.append(row[0])

      if str(shoe_type).lower() not in shoes:
        print('Please type shoe type as listed to purchase.')
        return
      else: 

        stock = globals()[shoe_type].check_stock() # checks stock of shoe
        if stock < quantity: # if not enough stock to fulfil order then order won't be accepted
          print('Not enough stock available.')
          print('Max order:', stock)
          return
          
        else: # if no problems with order then order will be fulfilled
          globals()[shoe_type].update_stock_from_order(quantity) 
          order_num = ''.join(['Od', str(next(ordernumiter))]) # order number generated. This is sequntial but starts with Od as to not be confused with customer id
          order_details = (order_num,
                          self.unique_id, 
                          datetime.now().strftime("%Y-%m-%d %H:%M:%S"), #order date generated
                          self.delivery_address, 
                          (datetime.now() + timedelta(hours=24)).strftime("%Y-%m-%d %H:%M:%S"), # latest delivery time calculated (now + 24 hours)
                          shoe_type,
                          quantity, 
                          (datetime.now() + timedelta(days=5)).strftime("%Y-%m-%d %H:%M:%S"),  # free returns date calculated (now + 5 days)
                          0, # order not returned
                          'N/A' # no return date
                          )
          cursor.execute('''
          INSERT INTO Orders
          VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
          ''',
          order_details
          ) # order added to Orders table in shoe_shop database
          connection.commit()

          # Bill printed for customer
          print('Order Recieved')
          print('')
          print('*' *49)
          print('*' *49)
          print('Bill', '-'*44)
          print('*' *49)
          print('')
          print('Order Number:', '.'*(34-len(order_num)), order_num)
          print('Item:', '.'*(42-len(shoe_type)), shoe_type)
          print('Quantity:', '.'*(38-len(str(quantity))), quantity)
          print('Price:', '.'*(39-len(str(globals()[shoe_type].price * quantity))), '£', globals()[shoe_type].price * quantity)
          print('*' *49)
          print('Order to be delivered by:', ' '*3, (datetime.now() + timedelta(hours=24)).strftime("%Y-%m-%d %H:%M:%S"))
          print('Free returns avialable until:', (datetime.now() + timedelta(days=5)).strftime("%Y-%m-%d %H:%M:%S"))
          print('')
          print('*' *49)
          print('*' *49)




  def return_product(self, order_num):
    if self.loggedin != True: # must be logged into account to make return
      print('Please login.')
      return

    else:
      connection = sqlite3.connect('shoe_shop.db')
      cursor = connection.cursor()
# below counts the number of returned orders within the past month
      cursor.execute('''
        SELECT COUNT(*) 
        FROM Orders
        WHERE Customer_ID = ?
        AND Returned = 1
        AND Returned_Date > ?      
      ''',
      (self.unique_id, (datetime.now() - relativedelta(months=1)).strftime("%Y-%m-%d %H:%M:%S")) #calculates the date 1 month ago
      )
      num_returned = cursor.fetchall()[0][0]
# below finds how many returns customer is allowed in a month
      cursor.execute('''
        SELECT max_returns_per_month
        FROM Customers
        WHERE unique_id = ?    
      ''',
      (self.unique_id, )
      )
      max_returns = cursor.fetchall()[0][0]
# if customer has already reached the max returns within the past month, application will not allow any more returns
      if num_returned >= max_returns: 
        print('Too many returns in past month.')
        print('Unable to return this order.')
        return
      
      else: # if max returns not reached, system checks to find order
        cursor.execute('''
        SELECT COUNT(*)
        FROM Orders
        WHERE Customer_ID = ?
        AND Order_Number = ?    
        ''',
        (self.unique_id, order_num)
        )

        if cursor.fetchall()[0][0] == 0:
          print('Unable to find order.')
          return
        
        else: #if order located checks to see that product hasnt already been returned
          cursor.execute('''
            SELECT Returned 
            FROM Orders
            WHERE Order_Number = ?
            ''',
            (order_num, ))          
          if 1 in cursor.fetchall()[0]:
            print('Order already returned')
            return
          
          else: # if no problem with making return find details of the order and updates the stock
            cursor.execute('''
              SELECT Product, Quantity
              FROM Orders
              WHERE Order_Number = ?
              ''',
              (order_num, )
            )
            stock_back = cursor.fetchall()[0]
            globals()[stock_back[0]].new_stock(stock_back[1])            
          # Also updates the order table to mark that the order has been returned and lists the return date
            cursor.execute('''
              UPDATE Orders
              SET Returned = 1, Returned_Date = ?
              WHERE Customer_ID = ?
              AND Order_Number = ?    
              ''',
            (datetime.now().strftime("%Y-%m-%d %H:%M:%S"), self.unique_id, order_num)
            )
            connection.commit()
            # Then finds the free return date from db
            cursor.execute('''
              SELECT Free_Return_Date
              FROM Orders
              WHERE Customer_ID = ?
              AND Order_Number = ? 
              ''',
              (self.unique_id, order_num)
            )
            free_return = datetime.strptime(cursor.fetchall()[0][0], "%Y-%m-%d %H:%M:%S")

            if free_return >= datetime.now():
              print('Return Successful!')
              print('No charge occured as within the free returns period')
            else:
              print('Return Successful!')
              print('£10 charge occured as not within the free returns period')


  def view_all_orders(self):
    if self.loggedin != True:
      print('Please login.')
      return

    else:
      table = pd.DataFrame()
      connection = sqlite3.connect('shoe_shop.db')
      cursor = connection.cursor()
      cursor.execute('''
        SELECT *
        FROM Orders
        WHERE Customer_ID = ?
        ''', # view_returned_orders methods has 'AND RETURNED = 1' in SQL Command here
        (self.unique_id, )
      )
      rows = cursor.fetchall()
      for row in rows:
        table = table.append(pd.Series(list(row)), ignore_index=True)
      table.columns = (['Order_Number',
                      'Customer_ID',
                      'Order_Date',
                      'Delivery_Address', 
                      'Latest_Delivery_Date',
                      'Product',
                      'Quantity',
                      'Free_Return_Date',
                      'Returned',
                      'Returned_Date'])
      table.Customer_ID = table.Customer_ID.astype('int')
      table.Returned = table.Returned.astype('int')
      return table

  def view_returned_orders(self):
    if self.loggedin != True:
      print('Please login.')
      return

    else:
      table = pd.DataFrame()
      connection = sqlite3.connect('shoe_shop.db')
      cursor = connection.cursor()
      cursor.execute('''
        SELECT *
        FROM Orders
        WHERE Customer_ID = ?
        AND RETURNED = 1
        ''',
        (self.unique_id, )
      )
      rows = cursor.fetchall()
      for row in rows:
        table = table.append(pd.Series(list(row)), ignore_index=True)
      table.columns = (['Order_Number',
                      'Customer_ID',
                      'Order_Date',
                      'Delivery_Address', 
                      'Latest_Delivery_Date',
                      'Product',
                      'Quantity',
                      'Free_Return_Date',
                      'Returned',
                      'Returned_Date'])
      table.Customer_ID = table.Customer_ID.astype('int')
      table.Returned = table.Returned.astype('int')
      return table



In [ ]:
class PremiumCustomer(Customer):

  def __init__(self, username, password, delivery_address, contact_info, bank_detail):
    super().__init__(username, password, delivery_address, contact_info, bank_detail)
    connection = sqlite3.connect('shoe_shop.db')
    cursor = connection.cursor()
    cursor.execute('''
      UPDATE Customers
      SET max_returns_per_month = 4
      WHERE username = ?
    ''',
    (username, )
    )
    connection.commit()
    connection.close()


In [ ]:
def create_account(username, password, delivery_address, contact_info, bank_detail, premium=False):
  if premium:
    return PremiumCustomer(username, password, delivery_address, contact_info, bank_detail)
  else:
    return Customer(username, password, delivery_address, contact_info, bank_detail)

In [ ]:
alex = create_account('alex.mcl@yahoo.com', 'Password1', '44 Loopland Rd', 7538933232, 4554)
george = create_account('george@yahoo.com', 'Password2', '16 Botanic Avenue', 32423849823, 3476347)
gaffer = create_account('gaffer2@yahoo.com', 'Password4', '10a Lockview Road', 3792374982, 455645, premium=True)

In [ ]:
alex1 = create_account('alex.mcl@yahoo.com', 'Password1', '44 Loopland Rd', 7538933232, 4554)

Username already registered please login.


In [ ]:
alex.login('george@yahoo.com', 'Password1')
george.login('george@yahoo.com', 'Password1')
george.login('george@yahoo.com', 'Password2')

Username not recognised.
Incorrect Password Entered.
Login successful


In [ ]:
george.search('clogs')
alex.search('slippers')

clogs .................... stock: 100
Please login to account.


In [ ]:
george.logout()
alex.logout()

Logout Successful
Not logged in


In [ ]:
george.login('george@yahoo.com', 'Password2')

Login successful


In [ ]:
george.search('a')
print('-'*38)
george.search('slip')


sandles .................. stock: 100
boaters .................. stock: 100
trainers ................. stock: 100
moccasins ................ stock: 100
--------------------------------------
slippers ................. stock: 200


In [ ]:
george.search('slippers', buy=True, quantity = 2)

slippers ................. stock: 200
Order Recieved

*************************************************
*************************************************
Bill --------------------------------------------
*************************************************

Order Number: ............................... Od0
Item: .................................. slippers
Quantity: ..................................... 2
Price: ..................................... £ 24
*************************************************
Order to be delivered by:     2022-05-26 10:21:54
Free returns avialable until: 2022-05-30 10:21:54

*************************************************
*************************************************


In [ ]:
alex.search('boat')
george.search('boat')
george.search('slip', buy=True)
print('-'*38)
george.search('slippers', buy=True)
print('-'*38)
george.search('slippers', buy=True, quantity = 2)

Please login to account.
boaters .................. stock: 100
slippers ................. stock: 198
Please type shoe type as listed to purchase.
--------------------------------------
slippers ................. stock: 198
please select quantity
--------------------------------------
slippers ................. stock: 198
Order Recieved

*************************************************
*************************************************
Bill --------------------------------------------
*************************************************

Order Number: ............................... Od1
Item: .................................. slippers
Quantity: ..................................... 2
Price: ..................................... £ 24
*************************************************
Order to be delivered by:     2022-05-26 10:21:54
Free returns avialable until: 2022-05-30 10:21:54

*************************************************
*************************************************


In [ ]:
gaffer.buy_shoes('clogs', quantity=10)
george.buy_shoes('clogs')

Please login.
Order Recieved

*************************************************
*************************************************
Bill --------------------------------------------
*************************************************

Order Number: ............................... Od2
Item: ..................................... clogs
Quantity: ..................................... 1
Price: ..................................... £ 23
*************************************************
Order to be delivered by:     2022-05-26 10:21:54
Free returns avialable until: 2022-05-30 10:21:54

*************************************************
*************************************************


In [ ]:
gaffer.login('gaffer2@yahoo.com', 'Password4')

Login successful


In [ ]:
gaffer.buy_shoes('sandles', 200)

Not enough stock available.
Max order: 100


In [ ]:
gaffer.buy_shoes('sandles', 2)

Order Recieved

*************************************************
*************************************************
Bill --------------------------------------------
*************************************************

Order Number: ............................... Od3
Item: ................................... sandles
Quantity: ..................................... 2
Price: ..................................... £ 40
*************************************************
Order to be delivered by:     2022-05-26 10:21:54
Free returns avialable until: 2022-05-30 10:21:54

*************************************************
*************************************************


In [ ]:
gaffer.buy_shoes('sand', 1)

Please type shoe type as listed to purchase.


In [ ]:
gaffer.return_product('Od2')

Unable to find order.


In [ ]:
alex.login('alex.mcl@yahoo.com', 'Password1')
alex.buy_shoes('boaters')

Login successful
Order Recieved

*************************************************
*************************************************
Bill --------------------------------------------
*************************************************

Order Number: ............................... Od4
Item: ................................... boaters
Quantity: ..................................... 1
Price: ..................................... £ 25
*************************************************
Order to be delivered by:     2022-05-26 10:21:54
Free returns avialable until: 2022-05-30 10:21:54

*************************************************
*************************************************


In [ ]:
connection = sqlite3.connect('shoe_shop.db')
cursor = connection.cursor()
cursor.execute('''
  UPDATE Orders
  SET Free_Return_Date = '2022-05-17 12:30:00'
  WHERE Order_Number = 'Od3'
''')
connection.commit()

In [ ]:
boaters.check_stock()

99

In [ ]:
gaffer.return_product('Od3')

Return Successful!
£10 charge occured as not within the free returns period


In [ ]:
boaters.check_stock()

99

In [ ]:
alex.return_product('Od3')

Unable to find order.


In [ ]:
alex.buy_shoes('boaters')
alex.buy_shoes('clogs')
alex.buy_shoes('stilettos')

Order Recieved

*************************************************
*************************************************
Bill --------------------------------------------
*************************************************

Order Number: ............................... Od5
Item: ................................... boaters
Quantity: ..................................... 1
Price: ..................................... £ 25
*************************************************
Order to be delivered by:     2022-05-26 10:21:54
Free returns avialable until: 2022-05-30 10:21:54

*************************************************
*************************************************
Order Recieved

*************************************************
*************************************************
Bill --------------------------------------------
*************************************************

Order Number: ............................... Od6
Item: ..................................... clogs
Quantity: .....

In [ ]:
alex.return_product('Od4')

Return Successful!
No charge occured as within the free returns period


In [ ]:
alex.return_product('Od5')

Return Successful!
No charge occured as within the free returns period


In [ ]:
alex.return_product('Od6')

Too many returns in past month.
Unable to return this order.


In [ ]:
gaffer.return_product('Od10')

Unable to find order.


In [ ]:
alex.view_all_orders()

,Order_Number,Customer_ID,Order_Date,Delivery_Address,Latest_Delivery_Date,Product,Quantity,Free_Return_Date,Returned,Returned_Date
0,Od4,0,2022-05-25 10:21:54,44 Loopland Rd,2022-05-26 10:21:54,boaters,1.0,2022-05-30 10:21:54,1,2022-05-25 10:21:54
1,Od5,0,2022-05-25 10:21:54,44 Loopland Rd,2022-05-26 10:21:54,boaters,1.0,2022-05-30 10:21:54,1,2022-05-25 10:21:54
2,Od6,0,2022-05-25 10:21:54,44 Loopland Rd,2022-05-26 10:21:54,clogs,1.0,2022-05-30 10:21:54,0,N/A
3,Od7,0,2022-05-25 10:21:54,44 Loopland Rd,2022-05-26 10:21:54,stilettos,1.0,2022-05-30 10:21:54,0,N/A


In [ ]:
alex.view_returned_orders()

,Order_Number,Customer_ID,Order_Date,Delivery_Address,Latest_Delivery_Date,Product,Quantity,Free_Return_Date,Returned,Returned_Date
0,Od4,0,2022-05-25 10:21:54,44 Loopland Rd,2022-05-26 10:21:54,boaters,1.0,2022-05-30 10:21:54,1,2022-05-25 10:21:54
1,Od5,0,2022-05-25 10:21:54,44 Loopland Rd,2022-05-26 10:21:54,boaters,1.0,2022-05-30 10:21:54,1,2022-05-25 10:21:54
